In [1]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
import requests
import xml.etree.ElementTree as ET
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import pandas as pd
data = pd.read_parquet('chatgpt_pubmed_summarization.parquet')

In [3]:
articles = pd.read_parquet('articles.parquet')

In [4]:
articles.head()

,article,abstract,record_id
0,gastric cancer is the second most frequent neo...,the typical symptoms of advanced cancer of the...,53
1,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...,204
2,beyond its role of cellular energy currency an...,release and reception of extracellular atp by ...,217
3,generalized anxiety disorder ( gad ) is a chro...,objective : to elucidate the consequences of b...,244
4,sterilization in orthodontics has repeatedly b...,background : sterilization techniques could af...,260


In [5]:
data

,record_id,prompt,ground_truth,mapped_ground_true,model_response,answer,mapped_answer,time
0,5238,"\nAs a medical expert, you are given a set of ...","during the last years , self - assembled organ...","during the last years , self - assembled organ...",This article explores the self-assembly of ord...,This article explores the self-assembly of ord...,This article explores the self-assembly of ord...,11-27-2023-22-00-38
1,912,"\nAs a medical expert, you are given a set of ...",aim . \n acute pancreatitis is an inflammatory...,aim . \n acute pancreatitis is an inflammatory...,Summary: This article examines the potential t...,Summary: This article examines the potential t...,Summary: This article examines the potential t...,11-27-2023-22-01-35
2,204,"\nAs a medical expert, you are given a set of ...",abstractthe incidence of pathological gambling...,abstractthe incidence of pathological gambling...,This article explores the relationship between...,This article explores the relationship between...,This article explores the relationship between...,11-27-2023-22-02-44
3,6074,"\nAs a medical expert, you are given a set of ...",purposepulmonary hypertension and exercise - i...,purposepulmonary hypertension and exercise - i...,This study investigated the relationship betwe...,This study investigated the relationship betwe...,This study investigated the relationship betwe...,11-27-2023-22-08-17
4,2253,"\nAs a medical expert, you are given a set of ...",an 810-nm diode laser was used to non - surgic...,an 810-nm diode laser was used to non - surgic...,This article emphasizes the importance of trea...,This article emphasizes the importance of trea...,This article emphasizes the importance of trea...,11-27-2023-22-09-00
...,...,...,...,...,...,...,...,...
94,584,"\nAs a medical expert, you are given a set of ...",oral cavity is an uncommon site for metastasis...,oral cavity is an uncommon site for metastasis...,The article discusses the rarity of oral metas...,The article discusses the rarity of oral metas...,The article discusses the rarity of oral metas...,11-27-2023-23-56-48
95,4990,"\nAs a medical expert, you are given a set of ...",acute appendicitis in children with acute leuk...,acute appendicitis in children with acute leuk...,In pediatric patients with acute leukemia unde...,In pediatric patients with acute leukemia unde...,In pediatric patients with acute leukemia unde...,11-27-2023-23-57-35
96,5201,"\nAs a medical expert, you are given a set of ...",eye injury is a significant disabling worldwid...,eye injury is a significant disabling worldwid...,The article highlights the prevalence and impa...,The article highlights the prevalence and impa...,The article highlights the prevalence and impa...,11-27-2023-23-59-29
97,1401,"\nAs a medical expert, you are given a set of ...",lead \n is a toxic heavy metal whose detoxific...,lead \n is a toxic heavy metal whose detoxific...,"Lead poisoning is a serious health issue, part...","Lead poisoning is a serious health issue, part...","Lead poisoning is a serious health issue, part...",11-28-2023-00-01-15


In [6]:
merged_df = pd.merge(data, articles, on='record_id')
merged_df.head()

,record_id,prompt,ground_truth,mapped_ground_true,model_response,answer,mapped_answer,time,article,abstract
0,5238,"\nAs a medical expert, you are given a set of ...","during the last years , self - assembled organ...","during the last years , self - assembled organ...",This article explores the self-assembly of ord...,This article explores the self-assembly of ord...,This article explores the self-assembly of ord...,11-27-2023-22-00-38,"during the last years , the self - assembly of...","during the last years , self - assembled organ..."
1,912,"\nAs a medical expert, you are given a set of ...",aim . \n acute pancreatitis is an inflammatory...,aim . \n acute pancreatitis is an inflammatory...,Summary: This article examines the potential t...,Summary: This article examines the potential t...,Summary: This article examines the potential t...,11-27-2023-22-01-35,acute pancreatitis is an inflammatory intra - ...,aim . \n acute pancreatitis is an inflammatory...
2,204,"\nAs a medical expert, you are given a set of ...",abstractthe incidence of pathological gambling...,abstractthe incidence of pathological gambling...,This article explores the relationship between...,This article explores the relationship between...,This article explores the relationship between...,11-27-2023-22-02-44,pathological gambling is defined in the curren...,abstractthe incidence of pathological gambling...
3,6074,"\nAs a medical expert, you are given a set of ...",purposepulmonary hypertension and exercise - i...,purposepulmonary hypertension and exercise - i...,This study investigated the relationship betwe...,This study investigated the relationship betwe...,This study investigated the relationship betwe...,11-27-2023-22-08-17,exacerbations of copd are associated with acce...,purposepulmonary hypertension and exercise - i...
4,2253,"\nAs a medical expert, you are given a set of ...",an 810-nm diode laser was used to non - surgic...,an 810-nm diode laser was used to non - surgic...,This article emphasizes the importance of trea...,This article emphasizes the importance of trea...,This article emphasizes the importance of trea...,11-27-2023-22-09-00,peri - implantitis is inflammation of the peri...,an 810-nm diode laser was used to non - surgic...


In [7]:
correct_summary = merged_df['mapped_ground_true'].tolist()
model_generated = merged_df['mapped_answer'].tolist()
articles = merged_df['article'].tolist()
print(len(correct_summary))
len(model_generated)

99


99

In [8]:
from rouge_score import rouge_scorer

In [ ]:
# Summarize osmosis transcripts and evaluate
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
for i, note in enumerate(correct_summary):
    
    # Evaluate BLEU score
    bleu_score = corpus_bleu([[correct_summary[i].split()]], [model_generated[i].split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = correct_summary[i]
    hypothesis = model_generated[i]
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    #print("Clinical Note:")
    #print(note)
    print("\nCorrect Summary:")
    print(correct_summary[i])
    print("\nGenerated Summary:")
    print(model_generated[i])
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

In [10]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.02440105008314705
Average ROUGE-1 Score: 0.3956064574181664
Average ROUGE-2 Score: 0.11900163562831803
Average ROUGE-L Score: 0.20788332878994376
Average METEOR Score: 0.26648276698219997


In [11]:
print(len(articles))
print(len(correct_summary))
print(len(model_generated))
print(len(meteor_scores))

99
99
99
99


In [12]:
data = {
    'articles': articles,
    'correct_summary': correct_summary,
    'model_generated': model_generated,
    'meteor_scores': meteor_scores
}

# Creating a DataFrame
df = pd.DataFrame(data)

# Exporting the DataFrame to an Excel file
df.to_excel('GPT3.5_output.xlsx', index=False)